In [79]:
import pandas as pd
import numpy as np

In [80]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import warnings
warnings.filterwarnings('ignore')

In [305]:
import os
import sys
path = os.path.abspath('..')
sys.path.append(path)

from essay_grader.text_feature import gen_text_feature
from essay_grader.title_text_similarity import gen_title_text_similarity_features

In [88]:
# # save the file to pickle file
#1
# filename = '../essay_grader/pickle_data/rf_model.pkl'
# pickle.dump(clf, open(filename, 'wb'))
#2
# import pickle
# df.to_pickle('../essay_grader/pickle_data/text_feature_df2.pkl')


# # load the model from disk
#1
# rf_model = pickle.load(open(filename, 'rb'))

#2
# import pickle
# with open('../essay_grader/pickle_data/text_feature_df.pkl', "rb") as file:
#     text_feature_df = pickle.load(file)

## load or save pickle data

In [306]:
data = pd.read_csv("../essay_grader/data/essay_data.csv")

In [307]:
%%time
all_feature_df = gen_text_feature(data)

CPU times: user 32.5 s, sys: 121 ms, total: 32.7 s
Wall time: 32.9 s


In [308]:
%%time
all_feature_df = gen_title_text_similarity_features(all_feature_df)

2020-08-26 16:33:12 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-08-26 16:33:12 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |

2020-08-26 16:33:12 INFO: Use device: cpu
2020-08-26 16:33:12 INFO: Loading: tokenize
2020-08-26 16:33:12 INFO: Loading: pos
2020-08-26 16:33:13 INFO: Loading: lemma
2020-08-26 16:33:13 INFO: Done loading processors!


CPU times: user 7.57 s, sys: 511 ms, total: 8.08 s
Wall time: 7 s


In [310]:
all_feature_df.head(1)

,text,year,name,title,score,level,title_name,vocab_richness,mean_word_syllable,word_count,...,avg_sentence_length,count_stopwords,flesch_reading_ease,freq_wok_words,freq_aok_words,freq_cliche_words,freq_argument_words,freq_absolute_words,title_key_word,frequency_title_words
0,the question is asking that in the same discip...,2017,"4, 5 - Est_Chen-fzn235-TOK_essay.docx",5,4,2,given access to the same facts how is it poss...,0.20936,1.540025,1525,...,16.223404,788,60.08,25,52,0,4,3,"[(give, v), (access, n), (fact, n), (disagreem...",89


In [312]:
all_feature_df.drop(columns=["name","title_key_word"],inplace=True)

In [313]:
all_feature_df.head(1)

,text,year,title,score,level,title_name,vocab_richness,mean_word_syllable,word_count,sentence_count,avg_sentence_length,count_stopwords,flesch_reading_ease,freq_wok_words,freq_aok_words,freq_cliche_words,freq_argument_words,freq_absolute_words,frequency_title_words
0,the question is asking that in the same discip...,2017,5,4,2,given access to the same facts how is it poss...,0.20936,1.540025,1525,94,16.223404,788,60.08,25,52,0,4,3,89


In [314]:
df =all_feature_df.copy()

In [37]:
# df["level"]= df["level"].map({1:1,2:2,3:3,4:4,5:4})

In [315]:
df.columns

Index(['text', 'year', 'title', 'score', 'level', 'title_name',
       'vocab_richness', 'mean_word_syllable', 'word_count', 'sentence_count',
       'avg_sentence_length', 'count_stopwords', 'flesch_reading_ease',
       'freq_wok_words', 'freq_aok_words', 'freq_cliche_words',
       'freq_argument_words', 'freq_absolute_words', 'frequency_title_words'],
      dtype='object')

In [316]:
numeric_features = ['vocab_richness', 'mean_word_syllable', 'word_count', 'sentence_count',
       'avg_sentence_length', 'count_stopwords', 'flesch_reading_ease',
       'freq_wok_words', 'freq_aok_words', 'freq_cliche_words',
       'freq_argument_words', 'freq_absolute_words', 'frequency_title_words']

In [317]:
text_features =["text", 
                "title_name",
               ]

In [318]:
X = df[text_features+
    numeric_features
]

Y = df['score']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

## PCA

In [328]:
scaler =MinMaxScaler()
text_df.iloc[:,6:] = scaler.fit_transform(text_df.iloc[:,6:])

In [329]:
from sklearn.decomposition import PCA
# df must be normalized first
def pca_features(df):
    pca = PCA(random_state=42)
    pca.fit(df.iloc[:,6:])
    Wt = pd.DataFrame(pca.components_)
    
    pca_result = pd.concat([pd.Series(df.iloc[:,6:].columns),abs(Wt[0])],axis=1)
    pca_result.columns = ['features','vector']
    pca_result_sorted = pca_result.sort_values(by='vector', ascending=False)
    feature_list = []
    for feature in pca_result_sorted[pca_result_sorted['vector']>0]['features']:
        feature_list.append(feature)
    return df[feature_list]

In [330]:
df= pca_features(text_df)

In [331]:
len(all_feature_df.iloc[:,6:].columns)

13

In [332]:
df.columns

Index(['freq_wok_words', 'flesch_reading_ease', 'vocab_richness',
       'mean_word_syllable', 'word_count', 'avg_sentence_length',
       'count_stopwords', 'sentence_count', 'freq_cliche_words',
       'freq_aok_words', 'freq_argument_words', 'freq_absolute_words'],
      dtype='object')

In [275]:
df.head(1)

,freq_wok_words,flesch_reading_ease,vocab_richness,mean_word_syllable,word_count,avg_sentence_length,count_stopwords,sentence_count,freq_cliche_words,freq_aok_words,freq_argument_words,freq_absolute_words
0,0.403226,0.82636,0.0,0.221303,0.698113,0.095829,0.792035,0.816901,0.0,0.632911,0.210526,0.230769


In [277]:
Y = text_df['score']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.25, random_state=42)

## Random Forest Model

In [323]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA


# define model

# model = XGBClassifier(max_depth=3, alpha = 10, n_estimators=300, learning_rate=0.1)
# max_depth=10, n_estimators=300 
model = RandomForestClassifier(random_state=42,max_depth=27, n_estimators=328 )

# define transform
numeric_transformer = Pipeline(steps=[
                ('scaler', MinMaxScaler()),
#                 ('pca', PCA(random_state=42,n_components=7)),
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer,
         df.columns,
#          numeric_features ,
        ),
    ], remainder='drop')

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

In [320]:
# clf.get_params().keys()

In [321]:
# param_grid = {
#       'classifier__n_estimators':(10,50,100,200,300),
#       'classifier__max_depth': (2,4,6,8,10,12),
# #       'preprocessor__num__pca__n_components':(5,6,7,8,9,10)
# }
# search = GridSearchCV(clf, param_grid, cv=5, scoring="accuracy").fit(X_train, y_train)
# print(search.best_params_)
# print(search.best_score_)

In [322]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
      'classifier__n_estimators':randint(1,1000),
      'classifier__max_depth': randint(1,30),
#       'preprocessor__num__pca__n_components':randint(1,12)
}
search = RandomizedSearchCV(clf, param_grid, cv=5, scoring="accuracy").fit(X_train, y_train)
print(search.best_params_)
print(search.best_score_)

{'classifier__max_depth': 27, 'classifier__n_estimators': 328}
0.2540106951871658


In [324]:
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report
print("Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))

Accuracy: 0.248
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00        14
           3       0.24      0.47      0.32        15
           4       0.31      0.42      0.36        38
           5       0.13      0.19      0.15        16
           6       0.27      0.16      0.20        19
           7       0.25      0.15      0.19        13
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         2

   micro avg       0.25      0.25      0.25       125
   macro avg       0.13      0.15      0.14       125
weighted avg       0.21      0.25      0.22       125



In [325]:
from collections import Counter
Counter(preds)

Counter({4: 52, 7: 8, 3: 29, 5: 23, 6: 11, 2: 2})

In [326]:
def eval_score(y_test, preds):
    total = len(y_test)
    hits = 0 
    for i,v in enumerate(preds):
        if y_test.iloc[i]-v <2 and y_test.iloc[i]-v >-2:
            hits +=1
    return hits/total


In [327]:
eval_score(y_test,preds)

0.608

In [156]:
str(clf.predict(final_df)[0])

'3'